# Let's do more with with web apps!

# Objectives
1. Talking to our own apis via requests
2. Creating interfaces using HTML
3. Integrating external APIs into the apps

Make sure the code from last week is in it's own file named [`myapp.py`](myapp.py)
```python
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/<name>")
def hello_name(name):
    return f"Hello {name}!"

if __name__ == '__main__':
    app.run(debug=True)
```

# How do we run the app?
1. Open a terminal:
`new-> terminal`
2. Navigate to the folder containing `myapp.py`
3. Because we add the `app.run()` function call, we can now start the app by running:
```bash
python myapp.py
```

# What does it say if it's running?
You should see output akin to:
```bash
 * Serving Flask app "myapp.py"
 * Environment: production
   WARNING: Do not use the development server in a production environment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 ```

# How can we test the app? 

* We know the url:  http://127.0.0.1:5000/
* It's a RESTful api -> use requests!


In [ ]:
import requests

r = requests.get("http://127.0.0.1:5000/")

In [ ]:
r

In [ ]:
r.text

# What about the <name\> view?

In [ ]:
r = requests.get("http://127.0.0.1:5000/can you hear me world?")

In [ ]:
r.text

Python and most python libraries support unicode since Python 3

In [ ]:
r = requests.get("http://127.0.0.1:5000/🙃")

In [ ]:
r.text

# Try your own!

# But interface/webpage?

* Optimal way is mostly use HTML/Javascript to talk to Python backend...
* We're gonna have Python make some of that HTML 

# Flask web apps generally need three things:
* .py files
* static - unchanging files (pictures, documents, etc)
* templates - templates for generated html files
More info: http://jonathansoma.com/tutorials/webapps/intro-to-flask/

# So where do we start?

In your `myapp` folder, create two folders:
1. static
2. templates

In templates, create a file called `hello.html` with the following content:
```html
<h1>Hello world!</h1>
```

# And how do we render that?
change
```python
@app.route("/")
def hello():
    return "Hello World!"
```
to 
```python
from flask import render_template

@app.route("/")
def hello():
    return render_template('hello.html')
```


r = requests.get("http://127.0.0.1:5000/")

In [ ]:
r.text

In [ ]:
from IPython.display import HTML, display

In [ ]:
HTML("http://127.0.0.1:5000")

The above should render as a webpage. Note, this is a terrible way to do web development and we're only doing it this way because of the limitations of the lab. You should do web development using plaintext files and by opening the browser to the web pages you're building.

# What about <name\>? That's the power of templates!

## Let's convert hello.html into a template:
change
```html
<h1>Hello world!</h1>
```
to 
```html
<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Hello {{ name }}!</h1>
{% else %}
  <h1>Hello, World!</h1>
{% endif %}
```

What's going on there? We're using Python logic to control the presentation. If name is present in the url, then replace `{{name}}` with `name`. Otherwise just print Hello World!

# And let's add on this second option for the same route:
change
```python
@app.route("/")
def hello():
    return render_template('hello.html')
```
to
```python
@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)
```

# Let's see what those two look like!

In [ ]:
HTML("http://127.0.0.1:5000/hello")

In [ ]:
HTML("http://127.0.0.1:5000/hello/hi")

# Try with your own URL:

# Let's make a thing...quiz? 
Further reading: https://radiusofcircle.blogspot.com/2016/03/making-quiz-website-with-python.html

# Let's start very very small - matching, 3 questions so we can test:
1. What is the programming language used in this class? Python, Assembly, Java, Lisp
2. How many homework assignments have been posted? 1, 3, 4, 2
3. What day of the week is it? Monday, Wednesday, Friday, Sunday

# What's a convenient way to encode the questions?

In [ ]:
#task: create a list of dictionaries storing {question:, options:[options], answer:, qid}

In [ ]:
questions = [{'question':'hihi', 'options':['a', 'b', 'c', 'd'], 'answer':'a', 'qid':1}, 
             {'question':'hi', 'options':['a', 'b', 'c', 'd'], 'answer':'b', 'qid':2}]

# Take your dictionary and store it in a data.py file

# Let's write a quiz function that randomly selects a question, asks the user to choose an answer, and checks it against the correct answer

In [ ]:
import random
def quiz(questions):
       
    print(q['question'])
    ans = input(f"choose 1:{q['options']}: ")
    if ans == q['answer']:
        return True
    return False

In [ ]:
quiz(questions)

# How do we make this an app?
It's a form, which is a post request because we're updating data

# We start with the template: how should this be displayed?


```html
<h1>Psych 31170 Quiz</h1>
<form action='/grade' method='POST'>
    <ol>
        {% for q in questions %}
            <li>{{q['question']}}</li>
            {% for o in q['options'] %}
                <input type='radio' value='{{o}}' name='{{loop.index }}'/>{{o}}
            {% endfor %}
        {% endfor %}
    </ol>
    <input type="submit" value="submit" />
</form>
```

Save the above in a questions.html file

In [ ]:
# what's that look like?
for i, q in enumerate(questions):
    print(q['question'])
    for o in q['options']:
        print(o, i)

The name attribute in the radio buttons keeps the group together, and the template code gets stored in a file called quiz.html in the templates folder.

# How do we talk to our quiz?

```python
import random 
from data import questions

@app.route('/quiz')
def quiz():
    #shuffle questions in place
    random.shuffle(questions)
    #shuffle options:
    for q in questions:
        random.shuffle(q['options'])
    return render_template('questions.html', questions=questions)
```

In [ ]:
HTML("http://127.0.0.1:5000/quiz")

# How do we check results? What happens on the submit?
```
<form action='/grade' method='POST'>
```

* request object - request->to form, response comes back
* form is an attribute on quest that returns form[name] where name is identified in the form elements:
```html
     <input type='radio' value='{{o}}' name='{{loop.index }}'/>
```
```python
@app.route('/grade', methods=['POST'])
def quiz_answers():
 correct = 0
 for q in questions:
    if q['answer'] == request.form[q['qid']]:
        corr

 return '<h1>Correct Answers: <u>'+str(correct)+'</u></h1>'
```

In [ ]:
HTML("http://127.0.0.1:5000/quiz")